In [1]:
import pandas as pd
import numpy as np
import warnings

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings('ignore')

In [2]:
def load_data(filepath):
    """Load dữ liệu từ file CSV"""
    print(f"Đang tải dữ liệu từ: {filepath}...")
    try:
        df = pd.read_csv(filepath)
        print(f"Đã tải thành công! Kích thước gốc: {df.shape}")
        return df
    except Exception as e:
        print(f"Lỗi tải file: {e}")
        return None


In [5]:
def clean_data(df):
    # 1. Chuẩn hóa tên cột (đổi dấu chấm thành gạch dưới về chữ thường)
    df.columns = df.columns.str.replace('.', '_').str.lower()
    print("- Đã chuẩn hóa tên cột (ví dụ: 'Order.ID' -> 'order_id').")

    # 2. Xóa cột rác (nếu có)
    junk_cols = ['row_id', '记录数', 'weeknum', 'market2']
    cols_to_drop = [c for c in junk_cols if c in df.columns]
    if cols_to_drop:
        df.drop(columns=cols_to_drop, inplace=True)
        print(f"- Đã xóa các cột thừa: {cols_to_drop}")

    # 3. Chuyển đổi định dạng ngày tháng
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['ship_date'] = pd.to_datetime(df['ship_date'])
    print("- Đã chuyển 'order_date' và 'ship_date' sang dạng Datetime.")

    # 4. Xử lý Logic Nghiệp vụ (Phần bạn muốn chuyển sang đây)
    # 4.1. Xóa đơn hàng có Ngày Ship < Ngày Đặt (Vô lý)
    invalid_dates = df[df['ship_date'] < df['order_date']]
    if not invalid_dates.empty:
        print(f"- ⚠️ Phát hiện {len(invalid_dates)} dòng lỗi (Ship < Order). Đang xóa...")
        df = df[df['ship_date'] >= df['order_date']]
    
    # 4.2. Xử lý trùng lặp (Business Key: Order ID + Product ID)
    # Group lại và cộng dồn các chỉ số số lượng
    # Lưu ý: Ở đây ta giả định đơn giản là xóa trùng lặp hoàn toàn trước
    before_dedup = len(df)
    df = df.drop_duplicates()
    after_dedup = len(df)
    if before_dedup > after_dedup:
        print(f"- Đã xóa {before_dedup - after_dedup} dòng trùng lặp hoàn toàn.")

    return df

In [ ]:
def feature_engineering(df):
    """
    Tạo các cột mới (Calculated Columns) phục vụ phân tích
    """
    print("\n--- BẮT ĐẦU FEATURE ENGINEERING ---")
    
    # 1.Hiệu suất giao hàng
    df['delivery_days'] = (df['ship_date'] - df['order_date']).dt.days
    # Giả định: Standard Class > 5 ngày là trễ 
    df['is_late'] = df['delivery_days'].apply(lambda x: 1 if x > 5 else 0)
    print("- Đã tạo cột: 'delivery_days', 'is_late'")

    # 2. Lợi nhuận
    # Tránh chia cho 0
    df['profit_margin'] = df.apply(lambda x: x['profit'] / x['sales'] if x['sales'] != 0 else 0, axis=1)
    df['is_loss_maker'] = df['profit'].apply(lambda x: 1 if x < 0 else 0)
    print("- Đã tạo cột: 'profit_margin', 'is_loss_maker'")

    # 3. Hành vi khách hàng 
    # Tìm ngày mua đầu tiên của mỗi khách hàng
    df['acquisition_date'] = df.groupby('customer_id')['order_date'].transform('min')
    df['cohort_month'] = df['acquisition_date'].dt.to_period('M').astype(str)
    print("- Đã tạo cột: 'acquisition_date', 'cohort_month' (cho Cohort Analysis)")

    # 4. Phân tích thời gian
    df['order_year'] = df['order_date'].dt.year
    df['order_month'] = df['order_date'].dt.month_name() # Tên tháng (January,...)
    df['order_quarter'] = df['order_date'].dt.to_period('Q').astype(str)
    print("- Đã tạo cột: 'order_year', 'order_month', 'order_quarter'")

    # 5. giảm giá
    def categorize_discount(x):
        if x == 0: return 'No Discount'
        elif x < 0.2: return 'Low (<20%)'
        elif x < 0.5: return 'Medium (<50%)'
        else: return 'High (>50%)'
    df['discount_level'] = df['discount'].apply(categorize_discount)
    print("- Đã tạo cột: 'discount_level'")

    return df

In [ ]:
def main():
    # Cấu hình đường dẫn
    INPUT_PATH = '../data/raw/superstore.csv' # File gốc
    OUTPUT_PATH = '../data/processed/superstore_final.csv' # File thành phẩm
    
    # 1. LOAD
    # Fallback cho chạy test tại chỗ
    try:
        raw_df = load_data(INPUT_PATH)
    except:
        raw_df = load_data('superstore.csv')

    if raw_df is not None:
        # 2. CLEAN
        clean_df = clean_data(raw_df)
        
        # 3. FEATURE ENGINEER
        final_df = feature_engineering(clean_df)
        
        # 4. REVIEW & EXPORT
        print("\n--- KẾT QUẢ CUỐI CÙNG ---")
        print(final_df.info())
        print(f"\nLoss Rate (Tỷ lệ đơn lỗ): {final_df['is_loss_maker'].mean():.1%}")
        
        print(f"\n Đang lưu file processed xuống: {OUTPUT_PATH}")
        # Tạo thư mục nếu chưa có, giả định folder đã tạo sẵn
        final_df.to_csv(OUTPUT_PATH, index=False)
        print(" Hoàn tất! Dữ liệu đã sẵn sàng cho Tableau.")

if __name__ == "__main__":
    main()

Đang tải dữ liệu từ: ../data/raw/superstore.csv...
Đã tải thành công! Kích thước gốc: (51290, 27)
- Đã chuẩn hóa tên cột (ví dụ: 'Order.ID' -> 'order_id').
- Đã xóa các cột thừa: ['row_id', '记录数', 'weeknum', 'market2']
- Đã chuyển 'order_date' và 'ship_date' sang dạng Datetime.

--- BẮT ĐẦU FEATURE ENGINEERING ---
- Đã tạo cột: 'delivery_days', 'is_late'
- Đã tạo cột: 'profit_margin', 'is_loss_maker'
- Đã tạo cột: 'acquisition_date', 'cohort_month' (cho Cohort Analysis)
- Đã tạo cột: 'order_year', 'order_month', 'order_quarter'
- Đã tạo cột: 'discount_level'

--- KẾT QUẢ CUỐI CÙNG ---
<class 'pandas.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   category          51290 non-null  str           
 1   city              51290 non-null  str           
 2   country           51290 non-null  str           
 3   customer_id       51290 non-null  s